## Урок 8. Рекуррентные нейронные сети RNN LSTM GRU


Задание
Данные берем отызывы за лето
На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Загрузка данных

import pandas as pd

df = pd.read_excel('/gdrive/MyDrive/Colab Notebooks/отзывы за лето.xls')
df.sample(15)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [ ]:
df.Content = df.Content.astype(str)

In [ ]:
# Предобработка текста

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

In [ ]:
df.sample(5)

,Rating,Content,Date
683,5,удобно,2017-08-13
5175,4,,2017-08-06
14302,1,пройти месяц проблема якобы незаводский прошив...,2017-07-24
11661,5,удобно совершать перевод разный способ комиссия,2017-07-28
14480,5,удобно обслуживание виза дороговатый,2017-07-23


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20659 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [ ]:
# Разделение выборки на тренировочную, тестовую и валидационную

from sklearn.model_selection import train_test_split

train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=49)

test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=49)

In [ ]:
train_df.to_csv('/gdrive/MyDrive/Colab Notebooks/train_df.csv')
test_df.to_csv('/gdrive/MyDrive/Colab Notebooks/test_df.csv')
val_df.to_csv('/gdrive/MyDrive/Colab Notebooks/val_df.csv')
test_val_df.to_csv('/gdrive/MyDrive/Colab Notebooks/test_val_df.csv')


In [ ]:
# Создание корпуса

#train_corpus = train_df['Content'].values # " ".join(train_df['Content'])
text_corpus_train = train_df['Content'].values
text_corpus_valid = val_df['Content'].values
text_corpus_test = test_df['Content'].values
#train_corpus = train_corpus.lower()

In [ ]:
text_corpus_train

array(['удобный приложение отлично работать',
       'root доступ многий суть перестать пользоваться приложение обновление сайт телефон meizu безопасность круто получаться недлить приложение рассчитать нестать ради сбербанк онлайн айфон sony покупать заработать звезда',
       'устраивать', ..., 'отличный приложение чётко работать',
       'работать', 'доступно качественно'], dtype=object)

In [ ]:
len(text_corpus_train)

14461

In [ ]:
num_classes = len(df['Rating'].unique())
num_classes

5

In [ ]:
# Импорт библиотек для построения сети

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes+1)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes+1)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes+1)
# y_train = train_df['Rating'].values
# y_val = val_df['Rating'].values

In [ ]:
y_train, len(y_train)

(array([[0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.]], dtype=float32), 14461)

In [ ]:
len(sequences_train)

14461

In [ ]:
y_train, y_val, len(y_train), len(y_val)

(array([[0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        ...,
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.]], dtype=float32),
 14461,
 3099)

In [ ]:
X_train, len(X_train), X_valid, len(X_valid)

(array([[   0,    0,    0, ...,    1,    5,    4],
        [   0,    0,    0, ..., 1219,  262,  144],
        [   0,    0,    0, ...,    0,    0,   22],
        ...,
        [   0,    0,    0, ...,    1,  174,    4],
        [   0,    0,    0, ...,    0,    0,    4],
        [   0,    0,    0, ...,    0,  150,  457]], dtype=int32),
 14461,
 array([[   0,    0,    0, ...,    0, 3597,   22],
        [   0,    0,    0, ...,  127, 2401, 9768],
        [   0,    0,    0, ...,   58,    9,  583],
        ...,
        [   0,    0,    0, ...,  592,  454,  658],
        [   0,    0,    0, ...,    0,    6,    1],
        [   0,    0,    0, ...,   71, 5845, 1579]], dtype=int32),
 3099)

## RNN

In [ ]:
word_count, training_length

(9829, 132)

In [ ]:
# RNN - Recurrent neural network - Рекуррентная нейронная сеть

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(132))
model.add(Dense(132, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss',patience =2,restore_best_weights=1)

model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 132, 30)           294870    
                                                                 
 masking_6 (Masking)         (None, 132, 30)           0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 132)               21516     
                                                                 
 dense_17 (Dense)            (None, 132)               17556     
                                                                 
 dropout_9 (Dropout)         (None, 132)               0         
                                                                 
 dense_18 (Dense)            (None, 6)                 798       
                                                                 
Total params: 334,740
Trainable params: 334,740
Non-tr

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 12s 420ms/step - loss: 1.1854 - accuracy: 0.6615 - val_loss: 0.9278 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 9s 332ms/step - loss: 0.9999 - accuracy: 0.7050 - val_loss: 0.8819 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 5s 196ms/step - loss: 0.8864 - accuracy: 0.7179 - val_loss: 1.1453 - val_accuracy: 0.7291
Epoch 4/10
26/26 [==============================] - 5s 203ms/step - loss: 0.8940 - accuracy: 0.7185 - val_loss: 0.7107 - val_accuracy: 0.7567
Epoch 5/10
26/26 [==============================] - 5s 204ms/step - loss: 0.7028 - accuracy: 0.7619 - val_loss: 0.6706 - val_accuracy: 0.7713
Epoch 6/10
26/26 [==============================] - 5s 200ms/step - loss: 0.6514 - accuracy: 0.7782 - val_loss: 0.6543 - val_accuracy: 0.7747
Epoch 7/10
26/26 [==============================] - 5s 202ms/step - loss: 0.6075 - accuracy: 0.7916 - val_loss: 0.6558 - val_accuracy: 0.7768
Epoch

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 17ms/step - loss: 0.6840 - accuracy: 0.7683


Test score: 0.6839525103569031
Test accuracy: 0.7683123350143433


In [ ]:
results = []

results.append(['RNN', score[0], score[1]])

## LSTM

In [ ]:
word_count, training_length


(9829, 132)

In [ ]:
# LSTM - Long short-term memory - Нейронная сеть долгой краткосрочной памяти

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(132, recurrent_dropout=0.2))
model.add(Dense(132, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))

In [ ]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 132, 30)           294870    
                                                                 
 masking_7 (Masking)         (None, 132, 30)           0         
                                                                 
 lstm_2 (LSTM)               (None, 132)               86064     
                                                                 
 dense_19 (Dense)            (None, 132)               17556     
                                                                 
 dropout_10 (Dropout)        (None, 132)               0         
                                                                 
 dense_20 (Dense)            (None, 6)                 798       
                                                                 
Total params: 399,288
Trainable params: 399,288
Non-t

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 18s 607ms/step - loss: 1.2609 - accuracy: 0.6736 - val_loss: 0.8705 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 14s 538ms/step - loss: 0.8625 - accuracy: 0.7149 - val_loss: 0.7475 - val_accuracy: 0.7560
Epoch 3/10
26/26 [==============================] - 14s 534ms/step - loss: 0.7502 - accuracy: 0.7516 - val_loss: 0.6875 - val_accuracy: 0.7623
Epoch 4/10
26/26 [==============================] - 16s 612ms/step - loss: 0.6671 - accuracy: 0.7692 - val_loss: 0.6614 - val_accuracy: 0.7761
Epoch 5/10
26/26 [==============================] - 16s 609ms/step - loss: 0.6166 - accuracy: 0.7860 - val_loss: 0.6553 - val_accuracy: 0.7802
Epoch 6/10
26/26 [==============================] - 15s 594ms/step - loss: 0.5782 - accuracy: 0.7964 - val_loss: 0.6608 - val_accuracy: 0.7823
Epoch 7/10
26/26 [==============================] - 14s 533ms/step - loss: 0.5453 - accuracy: 0.8107 - val_loss: 0.6774 - val_accuracy: 0.7747

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['LSTM', score[0], score[1]])

7/7 [==============================] - 0s 45ms/step - loss: 0.6882 - accuracy: 0.7641


Test score: 0.6882317066192627
Test accuracy: 0.7641174793243408


## GRU

In [ ]:
# GRU - Gated Recurrent Units - Управляемые рекуррентные блоки

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 132, 30)           294870    
                                                                 
 masking_8 (Masking)         (None, 132, 30)           0         
                                                                 
 gru_2 (GRU)                 (None, 64)                18432     
                                                                 
 dense_21 (Dense)            (None, 64)                4160      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 6)                 390       
                                                                 
Total params: 317,852
Trainable params: 317,852
Non-t

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 15s 492ms/step - loss: 1.4684 - accuracy: 0.6768 - val_loss: 0.9019 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 12s 477ms/step - loss: 0.9218 - accuracy: 0.7052 - val_loss: 0.7762 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 12s 474ms/step - loss: 0.8036 - accuracy: 0.7220 - val_loss: 0.7243 - val_accuracy: 0.7609
Epoch 4/10
26/26 [==============================] - 12s 473ms/step - loss: 0.7154 - accuracy: 0.7669 - val_loss: 0.6854 - val_accuracy: 0.7664
Epoch 5/10
26/26 [==============================] - 12s 481ms/step - loss: 0.6427 - accuracy: 0.7833 - val_loss: 0.6660 - val_accuracy: 0.7713
Epoch 6/10
26/26 [==============================] - 12s 473ms/step - loss: 0.5918 - accuracy: 0.7950 - val_loss: 0.6682 - val_accuracy: 0.7699
Epoch 7/10
26/26 [==============================] - 12s 479ms/step - loss: 0.5485 - accuracy: 0.8061 - val_loss: 0.6797 - val_accuracy: 0.7699

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['GRU', score[0], score[1]])

7/7 [==============================] - 0s 42ms/step - loss: 0.6936 - accuracy: 0.7641


Test score: 0.6936248540878296
Test accuracy: 0.7641174793243408


## CNN

In [ ]:
# CNN - Convolutional neural network - Сверточная нейронная сеть

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 132, 30)           294870    
                                                                 
 conv1d_4 (Conv1D)           (None, 130, 128)          11648     
                                                                 
 activation_4 (Activation)   (None, 130, 128)          0         
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_23 (Dense)            (None, 10)                1290      
                                                                 
 activation_5 (Activation)   (None, 10)                0         
                                                     

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 1s 18ms/step - loss: 1.5999 - accuracy: 0.5964 - val_loss: 1.2643 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 0s 11ms/step - loss: 1.2739 - accuracy: 0.6455 - val_loss: 0.9428 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 0s 11ms/step - loss: 1.0729 - accuracy: 0.6800 - val_loss: 0.7769 - val_accuracy: 0.7291
Epoch 4/10
26/26 [==============================] - 0s 11ms/step - loss: 0.9258 - accuracy: 0.6942 - val_loss: 0.7109 - val_accuracy: 0.7305
Epoch 5/10
26/26 [==============================] - 0s 11ms/step - loss: 0.8388 - accuracy: 0.7301 - val_loss: 0.6940 - val_accuracy: 0.7761
Epoch 6/10
26/26 [==============================] - 0s 11ms/step - loss: 0.7888 - accuracy: 0.7444 - val_loss: 0.6851 - val_accuracy: 0.7789
Epoch 7/10
26/26 [==============================] - 0s 11ms/step - loss: 0.7541 - accuracy: 0.7512 - val_loss: 0.6757 - val_accuracy: 0.7809
Epoch 8/10
26

In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['CNN', score[0], score[1]])

7/7 [==============================] - 0s 4ms/step - loss: 0.7133 - accuracy: 0.7738


Test score: 0.7132596373558044
Test accuracy: 0.7737979888916016


## RNN + CNN

In [ ]:
# RNN + CNN

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(SimpleRNN(132, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(132, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes+1, activation="softmax"))      


model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 132, 30)           294870    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 132, 132)          21516     
                                                                 
 conv1d_5 (Conv1D)           (None, 130, 132)          52404     
                                                                 
 conv1d_6 (Conv1D)           (None, 130, 64)           8512      
                                                                 
 flatten_1 (Flatten)         (None, 8320)              0         
                                                                 
 dropout_13 (Dropout)        (None, 8320)              0         
                                                                 
 dense_25 (Dense)            (None, 6)               

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 7s 232ms/step - loss: 1.1528 - accuracy: 0.6863 - val_loss: 0.7937 - val_accuracy: 0.7478
Epoch 2/10
26/26 [==============================] - 6s 223ms/step - loss: 0.8176 - accuracy: 0.7331 - val_loss: 0.7328 - val_accuracy: 0.7699
Epoch 3/10
26/26 [==============================] - 6s 224ms/step - loss: 0.6959 - accuracy: 0.7648 - val_loss: 0.6743 - val_accuracy: 0.7768
Epoch 4/10
26/26 [==============================] - 6s 225ms/step - loss: 0.6043 - accuracy: 0.7868 - val_loss: 0.6749 - val_accuracy: 0.7706
Epoch 5/10
26/26 [==============================] - 6s 226ms/step - loss: 0.5362 - accuracy: 0.8069 - val_loss: 0.7279 - val_accuracy: 0.7706


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['RNN + CNN', score[0], score[1]])

7/7 [==============================] - 0s 35ms/step - loss: 0.7087 - accuracy: 0.7615


Test score: 0.7086854577064514
Test accuracy: 0.7615360021591187


In [ ]:
results_df = pd.DataFrame(results, columns = ['Model', 'Test score', 'Test accuracy'])
results_df

,Model,Test score,Test accuracy
0,RNN,0.683953,0.768312
1,LSTM,0.688232,0.764117
2,GRU,0.693625,0.764117
3,CNN,0.713260,0.773798
4,RNN + CNN,0.708685,0.761536
